In [1]:
# %pip install -q pyspark delta-spark apache-airflow

In [ ]:
import os
import re
import time
import datetime
import logging
import traceback

from threading import Thread

from delta import *
from threading import Thread, Event
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col  # Добавьте этот импорт, если используете col()
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Конфигурация путей
base_path = "/Volumes/MacMini/Library/CloudStorage/GoogleDrive-dewuntain@gmail.com/Мой диск/2 Семестр/Курсовая СОБД/log_generator"

log_path = os.path.join(base_path, "logs", "app.log")
delta_table_path = os.path.join(base_path, "delta")

# Создание каталогов
os.makedirs(delta_table_path, exist_ok=True)

In [3]:
# Инициализация Spark с Delta Lake
builder = SparkSession.builder \
    .appName("Streaming Logs to Delta") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

25/06/02 18:01:47 WARN Utils: Your hostname, Mac-mini-admin.local resolves to a loopback address: 127.0.0.1; using 192.168.0.101 instead (on interface en8)
25/06/02 18:01:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Volumes/MacMini/.ivy2/cache
The jars for the packages stored in: /Volumes/MacMini/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e9f7db54-6292-4ed4-8e1b-98185268b6b4;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Volumes/MacMini/Library/CloudStorage/GoogleDrive-dewuntain@gmail.com/%d0%9c%d0%be%d0%b9%20%d0%b4%d0%b8%d1%81%d0%ba/2%20%d0%a1%d0%b5%d0%bc%d0%b5%d1%81%d1%82%d1%80/%d0%9a%d1%83%d1%80%d1%81%d0%be%d0%b2%d0%b0%d1%8f%20%d0%a1%d0%9e%d0%91%d0%94/log_generator/.venv/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found io.delta#delta-spark_2.12;3.3.1 in central
	found io.delta#delta-storage;3.3.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 80ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.1 from central in [default]
	io.delta#delta-storage;3.3.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-e9f7db54-6292-4ed4-8e1b-98185268b6b4
	confs: [default]
	0 artifacts copied, 3 already retrieved (0kB/3ms)
25/06/02 18:01:47 

In [4]:
# Функция для парсинга логов
def parse_logs(df):
  return df.select(
    regexp_extract('value', r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})', 1).alias("timestamp"),
    regexp_extract('value', r' - (INFO|WARNING|ERROR) - ', 1).alias("level"),
    regexp_extract('value', r'session_id=([^|]+)', 1).alias("session_id"),
    regexp_extract('value', r'traffic_source=([^|]+)', 1).alias("traffic_source"),
    regexp_extract('value', r'event_type=([^|]+)', 1).alias("event_type"),
    regexp_extract('value', r'product_id=([^|]+)', 1).alias("product_id"),
    regexp_extract('value', r'order_id=([^|]+)', 1).alias("order_id"),
    regexp_extract('value', r'search_query=([^|]+)', 1).alias("search_query"),
    regexp_extract('value', r'country=([^\|\n]+)', 1).alias("country")
  )

In [ ]:
def update_last_user_actions(parsed_df):

    try:
        # 1. Таблица последних действий пользователей
        if {"session_id", "event_type", "timestamp"}.issubset(parsed_df.columns):
            # Получаем последнее действие для каждой сессии
            last_actions = parsed_df.groupBy("session_id").agg(
                F.max("timestamp").alias("last_action_time"),
                F.first("event_type").alias("last_action_type"),
                F.first("country").alias("country"),
                F.first("traffic_source").alias("traffic_source")
            )
            
            # Сохраняем в Delta таблицу
            last_actions.write.format("delta").mode("overwrite") \
                .save(os.path.join(base_path, "last_user_actions"))

    except Exception as e:
        print(f"Ошибка при расчете метрик: {str(e)}")
        traceback.print_exc()

In [6]:
# Глобальный флаг для остановки
stop_event = Event()

offset_file = os.path.join(base_path, "offset.info")

# Функции для работы с offset
def read_offset():
    if os.path.exists(offset_file):
        with open(offset_file, 'r') as f:
            return int(f.read().strip())
    return 0

def write_offset(new_offset):
    with open(offset_file, 'w') as f:
        f.write(str(new_offset))
        
# Основной цикл обработки
def run_incremental_pipeline():
    print(f"Батч-пайплайн запущен.\nЛоги: {log_path}\nDelta таблица: {delta_table_path}")

    # while True:
    while not stop_event.is_set():
        try:
            # Чтение логов
            with open(log_path, 'r') as f:
                lines = f.readlines()

            # Получаем последнюю позицию
            offset = read_offset()
            new_lines = lines[offset:]
            print(time.strftime("%Y-%m-%d %H:%M:%S"))
            print(f"Считано {len(new_lines)} новых строк.")
            
            # Если нет новых строк, ждем
            if not new_lines:
                time.sleep(10)
                continue
            
            # Создаем DataFrame из новых строк
            rdd = spark.sparkContext.parallelize(new_lines).map(lambda x: (x,))
            df = spark.createDataFrame(rdd, ["value"])

            # Парсим логи
            parsed = parse_logs(df)
            parsed_clean = parsed.filter(col("event_type").isNotNull() & (col("event_type") != ""))
            
            # Сохранение сырых данных
            parsed_clean.write.format("delta").mode("append").save(delta_table_path)

            print(f"Сохранено {parsed_clean.count()} новых строк.")
            
            # Вычисление и сохранение метрик
            update_last_user_actions(parsed_clean)

            print(f"Обработано: {parsed_clean.count()} строк. Новые изменены.")

            # Обновляем offset
            write_offset(offset + len(new_lines))
            
        except Exception as e:
            print(f"Ошибка обработки: {e}")
            if stop_event.is_set():  # Если была запрошена остановка
                break
        
        # Ждем с возможностью прервать ожидание
        stop_event.wait(10)  # Вместо time.sleep(10)


# Запуск в фоновом потоке
batch_thread = Thread(target=run_incremental_pipeline, daemon=True)
batch_thread.start()

Батч-пайплайн запущен.
Логи: /Volumes/MacMini/Library/CloudStorage/GoogleDrive-dewuntain@gmail.com/Мой диск/2 Семестр/Курсовая СОБД/log_generator/logs/app.log
Delta таблица: /Volumes/MacMini/Library/CloudStorage/GoogleDrive-dewuntain@gmail.com/Мой диск/2 Семестр/Курсовая СОБД/log_generator/delta


2025-06-02 18:01:48
Считано 16 новых строк.
Сохранено 16 новых строк.
Обработано: 16 строк. Новые изменены.
2025-06-02 18:02:08
Считано 19 новых строк.
Сохранено 19 новых строк.
Обработано: 19 строк. Новые изменены.
2025-06-02 18:02:19
Считано 12 новых строк.
Сохранено 12 новых строк.
Обработано: 12 строк. Новые изменены.
2025-06-02 18:02:31
Считано 11 новых строк.
Сохранено 11 новых строк.
Обработано: 11 строк. Новые изменены.
2025-06-02 18:02:42
Считано 12 новых строк.
Сохранено 12 новых строк.
Обработано: 12 строк. Новые изменены.
2025-06-02 18:02:54
Считано 11 новых строк.
Сохранено 11 новых строк.
Обработано: 11 строк. Новые изменены.
2025-06-02 18:03:05
Считано 11 новых строк.
Сохранено 11 новых строк.
Обработано: 11 строк. Новые изменены.
2025-06-02 18:03:16
Считано 11 новых строк.
Сохранено 11 новых строк.
Обработано: 11 строк. Новые изменены.
2025-06-02 18:03:28
Считано 12 новых строк.
Сохранено 12 новых строк.
Обработано: 12 строк. Новые изменены.
2025-06-02 18:03:40
Считано 

In [ ]:
# Дожидаемся завершения инициализации Spark
time.sleep(5)
# Установка свойств Delta для поддержки Change Data Feed
spark.sql(f"""
    ALTER TABLE delta.`{delta_table_path}`
    SET TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

25/06/02 18:01:53 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/06/02 18:01:53 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
25/06/02 18:01:53 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
25/06/02 18:01:53 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
25/06/02 18:01:53 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


DataFrame[]

In [ ]:
# Остановка пайплайна
def stop_pipeline():
    print("Запрашиваю остановку пайплайна...")
    stop_event.set()  # Устанавливаем флаг остановки
    batch_thread.join(timeout=30)  # Ждем завершения (максимум 30 сек)
    if batch_thread.is_alive():
        print("Предупреждение: Поток не завершился корректно!")
    else:
        print("Пайплайн успешно остановлен")

# Для остановки выполните:
# stop_pipeline()

Как проверить результат?

In [ ]:
# Отображение истории изменений Delta таблицы
spark.sql(f"DESCRIBE HISTORY delta.`{delta_table_path}`").show(truncate=False)

+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                              |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------------------------------------------+------------+-----------------------------------+
|1276   |2025-06-02 18:44:34.351|NULL  |NULL    |WRITE    |{mode -> Append, partitionBy -> []}|NULL|NULL    |NULL     |1275       |Serializable  |true         |{numFiles -> 10, numO

In [58]:
from pyspark.sql.functions import max as max_
# Получение последней версии Delta таблицы
last_version = spark.sql(f"DESCRIBE HISTORY delta.`{delta_table_path}`") \
                   .select(max_("version")) \
                   .collect()[0][0]
print(f"Последняя версия Delta таблицы: {last_version}")

Последняя версия Delta таблицы: 1809


25/06/02 20:57:16 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [55]:
# Чтение изменений с использованием Change Data Feed
df = (
    spark.read
        .format("delta")
        .option("readChangeFeed", "true")  # правильно "readChangeFeed", а не "readChangeData"
        .option("startingVersion", last_version)
        .option("endingVersion", last_version)  # используем одну и ту же версию для start и end
        .load(delta_table_path)
        .filter(col("_change_type") == "insert")  # только вставки
)
df.show()

+-------------------+-------+--------------------+--------------+------------+----------+--------+------------+-------+------------+---------------+--------------------+
|          timestamp|  level|          session_id|traffic_source|  event_type|product_id|order_id|search_query|country|_change_type|_commit_version|   _commit_timestamp|
+-------------------+-------+--------------------+--------------+------------+----------+--------+------------+-------+------------+---------------+--------------------+
|2025-06-02 18:44:22|   INFO|949a7458-7e4c-49e...|      DESKTOP |     search |          |        |    пылесос |     BY|      insert|           1276|2025-06-02 18:44:...|
|2025-06-02 18:44:23|   INFO|5ddb4d49-312c-40b...|       MOBILE |     search |          |        |     камера |     RU|      insert|           1276|2025-06-02 18:44:...|
|2025-06-02 18:44:28|WARNING|98f61588-ed10-4d7...|      DESKTOP |   purchase |          |  60149 |            |     AR|      insert|           1276|20

25/06/02 19:24:03 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/06/02 19:24:03 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
25/06/02 19:24:03 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
25/06/02 19:24:03 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers


In [41]:
from delta.tables import DeltaTable
# Функция для вывода информации о таблице
def show_table_info(table_path, table_name):
    print(f"\n=== {table_name.upper()} ===")
    
    # История изменений
    print("\nИстория изменений (последние 5 версий):")
    spark.sql(f"DESCRIBE HISTORY delta.`{table_path}`").show(5, truncate=False)
    
    # Схема таблицы
    print("\nСхема таблицы:")
    spark.read.format("delta").load(table_path).printSchema()
    
    # Пример данных
    print("\nПример данных (5 строк):")
    spark.read.format("delta").load(table_path).limit(5).show()

In [42]:
# Вывод информации о таблицах
show_table_info(delta_table_path, "Основная таблица логов")


=== ОСНОВНАЯ ТАБЛИЦА ЛОГОВ ===

История изменений (последние 5 версий):
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                              |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+-----------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------------------------------------------+------------+-----------------------------------+
|1189   |2025-06-02 18:27:40.21 |NULL  |NULL    |WRITE    |{mode -> Append, partitionBy -> []}|NULL|NULL    

In [43]:
last_actions_path = os.path.join(base_path, "last_user_actions")
show_table_info(last_actions_path, "Таблица последних действий")


=== ТАБЛИЦА ПОСЛЕДНИХ ДЕЙСТВИЙ ===

История изменений (последние 5 версий):
+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation|operationParameters                   |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                            |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------+--------------------------------------+----+--------+---------+-----------+--------------+-------------+------------------------------------------------------------+------------+-----------------------------------+
|1185   |2025-06-02 18:27:40.7  |NULL  |NULL    |WRITE    |{mode -> Overwrite, partitionBy -> []}|NUL

In [45]:
# Дополнительные полезные запросы
print("\nДополнительная информация:")
print(f"Последняя версия основной таблицы: {DeltaTable.forPath(spark, delta_table_path).history(1).select('version').collect()[0][0]}")
print(f"Количество записей в основной таблице: {spark.read.format('delta').load(delta_table_path).count()}")


Дополнительная информация:
Последняя версия основной таблицы: 1190
Количество записей в основной таблице: 13373
